In [ ]:
import pandas as pd
import numpy as np
import re

In [ ]:
gender_df = pd.read_csv('/content/drive/MyDrive/Carnegie Mellon University/15-688 Practical Data Science/Cleaned tables/Gender.csv')
gender_df.head()

,State,State_Abbr,Year,Gender,Population
0,Alabama,AL,1991,Female,2134265
1,Alabama,AL,1991,Male,1964891
2,Alabama,AL,1992,Female,2160659
3,Alabama,AL,1992,Male,1993355
4,Alabama,AL,1993,Female,2190193


In [ ]:
gender_pivot_df = pd.pivot_table(gender_df, columns='Gender', index=['State','State_Abbr','Year'])
gender_pivot_df.head()

Population         
Gender                      Female     Male
State   State_Abbr Year                    
Alabama AL         1991    2134265  1964891
                   1992    2160659  1993355
                   1993    2190193  2024009
                   1994    2212954  2047275
                   1995    2230497  2066303

In [ ]:
gender_pivot_df['Female_per_male'] = gender_pivot_df[('Population', 'Female')]/gender_pivot_df[('Population',   'Male')]
gender_pivot_df.head()

Population          Female_per_male
Gender                      Female     Male                
State   State_Abbr Year                                    
Alabama AL         1991    2134265  1964891        1.086200
                   1992    2160659  1993355        1.083931
                   1993    2190193  2024009        1.082106
                   1994    2212954  2047275        1.080927
                   1995    2230497  2066303        1.079463

In [ ]:
gender_cl_df = pd.DataFrame(
    gender_pivot_df['Female_per_male']
    .reset_index().set_index(['State_Abbr','State','Year'])
)
gender_cl_df.head()

Female_per_male
State_Abbr State   Year                 
AL         Alabama 1991         1.086200
                   1992         1.083931
                   1993         1.082106
                   1994         1.080927
                   1995         1.079463

In [ ]:
housing_df = pd.read_csv('/content/drive/MyDrive/Carnegie Mellon University/15-688 Practical Data Science/Cleaned tables/Housing.csv')
housing_df.head()

,index,State,Abbreviation,FIPS,Year,Annual Change (%),HPI,HPI with 1990 base,HPI with 2000 base
0,16,Alabama,AL,1,1991,2.29,204.57,102.29,74.57
1,17,Alabama,AL,1,1992,2.72,210.12,105.07,76.60
2,18,Alabama,AL,1,1993,3.19,216.83,108.43,79.04
3,19,Alabama,AL,1,1994,3.37,224.14,112.08,81.71
4,20,Alabama,AL,1,1995,4.14,233.43,116.72,85.09


In [ ]:
housing_df['FIPS'].unique()

array([ 1,  2,  4,  5,  6,  8,  9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20,
       21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37,
       38, 39, 40, 41, 42, 44, 45, 46, 47, 48, 49, 50, 51, 53, 54, 55, 56])

In [ ]:
housing_cl_df = pd.DataFrame(
    housing_df[['State','Abbreviation','Year','HPI']]
    .rename(columns={'Abbreviation':'State_Abbr'})
    .set_index(['State_Abbr','State','Year'])
)
housing_cl_df.head()

HPI
State_Abbr State   Year        
AL         Alabama 1991  204.57
                   1992  210.12
                   1993  216.83
                   1994  224.14
                   1995  233.43

In [ ]:
pop_dens_df = pd.read_csv('/content/drive/MyDrive/Carnegie Mellon University/15-688 Practical Data Science/Cleaned tables/population_density.csv')
pop_dens_df.head()

,States,Pop_per_km2,Perc_asian,num_asians_per_km2
0,District of Columbia,4251,4.5,192
1,Guam,314,32.2,102
2,Northern Mariana Islands,118,49.9,59
3,Hawaii,86,57.4,50
4,New Jersey,470,9.0,43


In [ ]:
state_abbr_df = housing_cl_df.reset_index()[['State','State_Abbr']].drop_duplicates().set_index('State')
state_abbr_df.head()

,State_Abbr
State,
Alabama,AL
Alaska,AK
Arizona,AZ
Arkansas,AR
California,CA


In [ ]:
pop_dens_df = pd.merge(pop_dens_df, state_abbr_df, left_on='States', right_on='State')
pop_dens_df.head()

,States,Pop_per_km2,Perc_asian,num_asians_per_km2,State_Abbr
0,District of Columbia,4251,4.5,192,DC
1,Hawaii,86,57.4,50,HI
2,New Jersey,470,9.0,43,NJ
3,Massachusetts,336,6.0,21,MA
4,Maryland,238,6.4,16,MD


In [ ]:
pop_dens_cl_df = pd.DataFrame(
    pop_dens_df
    [['State_Abbr','States','num_asians_per_km2']]
    .rename(columns={'States':'State'})
    .set_index(['State_Abbr','State'])
)
pop_dens_cl_df.head()

,,num_asians_per_km2
State_Abbr,State,
DC,District of Columbia,192
HI,Hawaii,50
NJ,New Jersey,43
MA,Massachusetts,21
MD,Maryland,16


In [ ]:
crime_df = pd.read_csv('/content/drive/MyDrive/Carnegie Mellon University/15-688 Practical Data Science/Cleaned tables/FBI_hate_crime.csv')
crime_df.head()

,INCIDENT_ID,DATA_YEAR,STATE_ABBR,STATE_NAME,INCIDENT_DATE,TOTAL_OFFENDER_COUNT,OFFENDER_RACE,VICTIM_COUNT,OFFENSE_NAME,LOCATION_NAME,BIAS_DESC,VICTIM_TYPES
0,33,1991,AZ,Arizona,13-NOV-91,1,White,1,Intimidation,Highway/Road/Alley/Street/Sidewalk,Anti-Asian,Individual
1,3010,1991,AZ,Arizona,07-DEC-91,1,Unknown,1,Intimidation,Restaurant,Anti-Asian,Individual
2,51,1991,CA,California,16-AUG-91,2,White,2,Aggravated Assault,Parking/Drop Lot/Garage,Anti-Asian,Individual
3,77,1991,CO,Colorado,01-AUG-91,1,White,1,Aggravated Assault,Residence/Home,Anti-Asian,Individual
4,95,1991,CO,Colorado,29-JUN-91,1,White,1,Destruction/Damage/Vandalism of Property,Other/Unknown,Anti-Asian,Individual


In [ ]:
crime_cl_df = pd.DataFrame(
    crime_df.groupby(['STATE_ABBR','STATE_NAME','DATA_YEAR'])
    .agg('count')['INCIDENT_ID']
    .reset_index()
    .rename(columns={'STATE_ABBR':'State_Abbr','STATE_NAME':'State','DATA_YEAR':'Year','INCIDENT_ID':'num_crimes'})
    .set_index(['State_Abbr','State','Year'])
)

crime_cl_df.head()

num_crimes
State_Abbr State  Year            
AK         Alaska 1993           2
                  1995           1
                  1997           1
                  2003           1
                  2004           1

In [ ]:
age_df = pd.read_csv('/content/drive/MyDrive/Carnegie Mellon University/15-688 Practical Data Science/Cleaned tables/Age.csv')
age_df.head()

,State,State_Abbr,Year,Age_Group_Code,Population
0,Alabama,AL,1991,1,60359
1,Alabama,AL,1991,1-4,229611
2,Alabama,AL,1991,10-14,302899
3,Alabama,AL,1991,15-19,310490
4,Alabama,AL,1991,20-24,319230


In [ ]:
age_groups = age_df['Age_Group_Code'].unique()
age_groups = sorted(age_groups, key=lambda x: int(re.match('^([0-9]+)', x)[0]))
print(age_groups)

['1', '1-4', '5-9', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40-44', '45-49', '50-54', '55-59', '60-64', '65-69', '70-74', '75-79', '80-84', '85+']


In [ ]:
age_pivot = pd.pivot_table(age_df, index=['State','State_Abbr','Year'], columns=['Age_Group_Code'])
age_cl_df = age_pivot.copy()
age_cl_df.columns = age_cl_df.columns.get_level_values(1)
age_cl_df.columns.name = None
age_cl_df = age_cl_df[age_groups]
age_cl_df['sum'] = age_cl_df[age_groups].apply(lambda x: sum(x), axis=1)
for grp in age_groups:
  age_cl_df[grp] = age_cl_df[[grp,'sum']].apply(lambda x: x[0]/x[1], axis=1)
age_cl_df.drop('sum', axis=1, inplace=True)
age_cl_df.head()

1       1-4  ...     80-84       85+
State   State_Abbr Year                      ...                    
Alabama AL         1991  0.014725  0.056014  ...  0.016731  0.012333
                   1992  0.014582  0.056390  ...  0.016862  0.012735
                   1993  0.014166  0.056847  ...  0.017159  0.013041
                   1994  0.013781  0.056895  ...  0.017288  0.013441
                   1995  0.013533  0.056634  ...  0.017449  0.013835

[5 rows x 19 columns]

In [ ]:
edu_pov_df = pd.read_csv('/content/drive/MyDrive/Carnegie Mellon University/15-688 Practical Data Science/Cleaned tables/combined.csv')
edu_pov_df.head()

,Unnamed: 0,State,Year,Total poor,Below poverty,Percent of poor below poverty,Percent HS or higher,Percent Bach or higher
0,0,Alabama,1991,"4,178",786,18.8,67.3,15.4
1,1,Alabama,1992,"4,226",729,17.3,NaN,NaN
2,2,Alabama,1993,"4,168",725,17.4,76.0,14.6
3,3,Alabama,1994,"4,299",704,16.4,72.5,15.2
4,4,Alabama,1995,"4,396",882,20.1,74.4,17.3


In [ ]:
edu_pov_cl_df = (
    edu_pov_df[['State','Year','Total poor','Percent HS or higher','Percent Bach or higher']].copy()
    .set_index('State')
)
edu_pov_cl_df = pd.merge(edu_pov_cl_df, state_abbr_df, left_index=True, right_index=True)
edu_pov_cl_df.fillna(method='backfill', inplace=True)
edu_pov_cl_df.reset_index(inplace=True)
edu_pov_cl_df = pd.merge(edu_pov_cl_df, pop_dens_df, left_on='State_Abbr', right_on='State_Abbr')
edu_pov_cl_df['Total poor'] = edu_pov_cl_df['Total poor'].apply(lambda x: x.replace(',',''))
edu_pov_cl_df['Total poor'] = edu_pov_cl_df['Total poor'].astype('int')
edu_pov_cl_df['Norm_poor'] = edu_pov_cl_df['Total poor']/edu_pov_cl_df['Pop_per_km2']
edu_pov_cl_df = edu_pov_cl_df.reset_index().set_index(['State_Abbr','State','Year'])
edu_pov_cl_df = edu_pov_cl_df[[
                               'Percent HS or higher',
                               'Percent Bach or higher',
                               'Norm_poor'
]]
edu_pov_cl_df.head()

Percent HS or higher  ...   Norm_poor
State_Abbr State   Year                        ...            
AL         Alabama 1991                  67.3  ...  112.918919
                   1992                  76.0  ...  114.216216
                   1993                  76.0  ...  112.648649
                   1994                  72.5  ...  116.189189
                   1995                  74.4  ...  118.810811

[5 rows x 3 columns]

In [ ]:
merge_df = (
    pd.merge(housing_cl_df, crime_cl_df, left_index=True, right_index=True, how='outer')
)
merge_df['num_crimes'] = merge_df['num_crimes'].fillna(0).astype('int')

merge_df = (
    pd.merge(merge_df, gender_cl_df, left_index=True, right_index=True)
)

merge_df = (
    pd.merge(merge_df, pop_dens_cl_df, left_index=True, right_index=True)
)

merge_df = (
    pd.merge(merge_df, edu_pov_cl_df, left_index=True, right_index=True)
)

merge_df = (
    pd.merge(merge_df, age_cl_df, left_index=True, right_index=True)
)

merge_df.sort_values(['num_crimes'], ascending=False, inplace=True)
merge_df.head()

HPI  num_crimes  ...     80-84       85+
State_Abbr State      Year                      ...                    
CA         California 1997  422.33         160  ...  0.014473  0.011779
                      1996  411.04         153  ...  0.014328  0.011519
                      1995  411.40         142  ...  0.014035  0.011254
                      1998  456.42         135  ...  0.014502  0.011990
                      1999  487.38         126  ...  0.014715  0.012337

[5 rows x 26 columns]

In [ ]:
merge_df.to_csv('/content/drive/MyDrive/Carnegie Mellon University/15-688 Practical Data Science/Cleaned tables/merged.csv')